In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib
import matplotlib.pyplot as plt
import gc
from captum.attr import *
import quantus
from torch.utils.data import DataLoader
import gc
import torchvision.transforms as transforms
import torchvision.models as models

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# the validation transforms
valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

In [5]:
images = '/home/db1702/Downloads/imagenet-mini/train/'

In [6]:
def load_imagenet_model():
    model = models.resnet50(pretrained=True).to(device)
    model.to('cuda')
    model.train(False)
    return model

In [7]:
normal_model = load_imagenet_model()
normal_model.to(device)
normal_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [8]:
#get dataset
test = torchvision.datasets.ImageFolder(images, transform=valid_transform)
test_loader = DataLoader(test, shuffle=True, batch_size = 2)

# For given adversarial images and benign images, collect metrics of feature attribution sensitivity and model prediction sensitivity. Save in csv that will be used for inspecting detection performance. 

In [9]:
import pandas as pd

In [10]:
def make_noise(x_batch, y_batch, spread):
    new_x_batch = []
    for x in x_batch:
        x = x.data.cpu().numpy()
        stdev = spread * (np.max(x)-np.min(x))
        noise = np.random.normal(0, stdev, x.shape).astype(np.float32)
        x_plus_noise = x + noise
        x_plus_noise = np.clip(x_plus_noise, 0, 1)
        x_plus_noise = torch.from_numpy(x_plus_noise).cpu()
        new_x_batch.append(x_plus_noise)
    new_batch = torch.stack(new_x_batch).to(device)
    return new_batch

In [11]:
# Define uniform noise function
def add_uniform_noise(image):
    # Generate uniform noise with mean 0 and standard deviation 25
    noise = np.random.uniform(low=-0.5, high=0.5, size=image.shape).astype(np.float32)
    noisy_image = np.clip(image + noise, 0, 1).astype(np.uint8)
    return noisy_image

def uniform_noise(x_batch, y_batch): 
    # Convert batch of images to numpy array
    images = x_batch.detach().cpu().numpy().transpose(0, 2, 3, 1) * 1.0
    # Add Poisson noise to each image in the batch
    noisy_images = [add_uniform_noise(image) for image in images]
    # Convert noisy images back to Tensor format
    noisy_inputs = torch.from_numpy(np.array(noisy_images).transpose(0, 3, 1, 2) / 1.0).float()
    return noisy_inputs.to(device)


In [12]:
def compute_metrics_benign(adv_path, normal_model): 
    
    print("Computing metrics for {} for benign")
    
    npobj = np.load(adv_path)
    adaptive_image = npobj['b_images']
    adaptive_label = npobj['b_labels']
    
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    
    images, labels = torch.from_numpy(adaptive_image), torch.from_numpy(adaptive_label)
    #images, labels = images.to(device), labels.to(device)
    
    end = len(adaptive_label)
    if end > 1000:
        end = 1000
    
    for i in range(0, end, 2):
        
        images_adv, y_pred_adv = images[i:i+2], labels[i:i+2]
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        x_logits = normal_model(images_adv)
        gc.collect()
        torch.cuda.empty_cache()
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.15)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.25)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.35)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(images_adv, y_pred_adv)
        uniform_logits = normal_model(uniform_noisy_images)
        
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1) 
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_uniform.append(c)
        
        
    df = pd.DataFrame([
            
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
        attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
        logit_uniform
    ], index = [
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution",
        "uniform attr",
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
        "uniform logit"
                    ])
            
    path = "adaptive_Benign.csv"
    df.to_csv(path)

In [13]:
def compute_metrics_adv(adv_path, normal_model): 
    
    print("Computing metrics for {} for adv")
    npobj = np.load(adv_path)
    adaptive_image = npobj['a_images']
    adaptive_label = npobj['a_labels']
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    
    images, labels = torch.from_numpy(adaptive_image), torch.from_numpy(adaptive_label)
    #images, labels = images.to(device), labels.to(device)
    
    end = len(adaptive_label)
    if end > 1000:
        end = 1000
    
    for i in range(0, end, 2):
        
        images_adv, y_pred_adv = images[i:i+2], labels[i:i+2]
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        x_logits = normal_model(images_adv)
        gc.collect()
        torch.cuda.empty_cache()
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.15)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.25)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.35)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(images_adv, y_pred_adv)
        uniform_logits = normal_model(uniform_noisy_images)
        
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1) 
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_uniform.append(c)
        
        
    df = pd.DataFrame([
            
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
        attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
    logit_uniform], index = [
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution",
        "uniform attr",
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
        "uniform logit"
                    ])
            
    path = "adaptive_Adv.csv"
    df.to_csv(path)

In [14]:
def compute_TPR(adv1, a, b, adv2, c, d): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value1, value2 in zip(adv1, adv2): 
        if value1<a or value1>b:
            TP += 1
        else:
            if value2<c or value2>d:
                TP+=1
            else: 
                FN+=1
    
    return (TP/(TP+FN))*100

In [15]:
def compute_FPR(ap2a, k, l, ap2b, m, n): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value6, value7 in zip(ap2a,ap2b):
        if value6<k or value6>l:
            FP +=1
        else:
            if value7<m or value7>n:
                FP +=1

    return (FP/(len(ap2a)))*100

In [16]:
import sklearn
from sklearn.metrics import roc_auc_score

In [30]:
def return_auc(adv_path, model):
    #logitgaussian3
#     k=[1810,2510,2810,3000, 3200,3500, 6000]
#     l=[6700,6700,6700,6700,6700,6700, 6700]
    
    k=[1023, 1300,1490,1580, 1600, 1700, 1800, 1900, 2100]
    l=[2970, 2970,2970,2970, 2970, 2970, 2970, 2970, 2970]
    
    m=[1340, 1900, 2200, 2400,  2800, 3000, 3400, 5000,7000 ]
    n=[10894, 10894, 10894,10894,10894,10894, 10894, 10894, 10894]

    
    #attrgaussian3
#     m=[1400,1750,1900, 2300, 3200, 3500, 6000]
#     n=[6600,6600,6600,6600,6600,6600,6600]

    
    compute_metrics_benign(adv_path, model)
    compute_metrics_adv(adv_path, model)
    df_cifar = pd.read_csv("adaptive_Benign.csv")
    attr_gaussian3 = df_cifar.iloc[2].values.flatten().tolist()[1:]
    logit_gaussian3 = df_cifar.iloc[6].values.flatten().tolist()[1:]
        
    fpr_results =[]
    for t1,t2,t3,t4 in zip(k,l,m,n):
        FPR = compute_FPR(logit_gaussian3, t1,t2, attr_gaussian3,t3,t4)
        fpr_results.append(FPR/100)
        
    df_pgd_eps1 = pd.read_csv("adaptive_Adv.csv")
    attr_gaussian3_eps1 = df_pgd_eps1.iloc[2].values.flatten().tolist()[1:]
    logit_gaussian3_eps1 = df_pgd_eps1.iloc[6].values.flatten().tolist()[1:]
    
    tpr_results =[]
    for t1,t2,t3,t4 in zip(k,l,m,n):
        TPR = compute_TPR(logit_gaussian3_eps1, t1,t2, attr_gaussian3_eps1,t3,t4)
        tpr_results.append(TPR/100)
    return(sklearn.metrics.auc(fpr_results, tpr_results), tpr_results, fpr_results)

# FGSM

In [31]:

adv_path1 = 'adv samples/IMAGENET/ResNet50/FGSM/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/IMAGENET/ResNet50/FGSM/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/IMAGENET/ResNet50/FGSM/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/IMAGENET/ResNet50/FGSM/0.25098039215686274eps.npz'

In [32]:
auc, tpr, fpr = return_auc(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.6485255,
 [0.01, 0.09300000000000001, 0.281, 0.425, 0.545, 0.69, 0.847, 0.985, 1.0],
 [0.005, 0.044, 0.133, 0.215, 0.319, 0.462, 0.679, 0.93, 0.996])

In [33]:
print(return_auc(adv_path2, normal_model))
print('----')
print(return_auc(adv_path3, normal_model))
print('----')
print(return_auc(adv_path4, normal_model))

Computing metrics for {} for benign
Computing metrics for {} for adv
(0.7535475, [0.01, 0.13, 0.3990000000000001, 0.584, 0.728, 0.858, 0.945, 0.998, 1.0], [0.001, 0.042, 0.127, 0.222, 0.32899999999999996, 0.46, 0.662, 0.929, 0.995])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.861404, [0.017, 0.282, 0.616, 0.787, 0.922, 0.9739999999999999, 0.996, 1.0, 1.0], [0.002, 0.038, 0.121, 0.23200000000000004, 0.337, 0.46, 0.652, 0.931, 0.998])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.9507700000000001, [0.087, 0.721, 0.926, 0.977, 0.998, 0.9990000000000001, 1.0, 1.0, 1.0], [0.004, 0.032, 0.132, 0.22, 0.325, 0.467, 0.664, 0.926, 0.995])


# PGD

In [34]:
adv_path1 = 'adv samples/IMAGENET/ResNet50/PGD/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/IMAGENET/ResNet50/PGD/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/IMAGENET/ResNet50/PGD/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/IMAGENET/ResNet50/PGD/0.25098039215686274eps.npz'

In [35]:
auc, tpr, fpr = return_auc(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.9840949999999999,
 [0.988, 0.988, 0.988, 0.988, 0.992, 0.992, 0.992, 0.996, 0.998],
 [0.004, 0.038, 0.124, 0.199, 0.297, 0.442, 0.626, 0.907, 0.996])

In [36]:
print(return_auc(adv_path2, normal_model))
print('----')
print(return_auc(adv_path3, normal_model))
print('----')
print(return_auc(adv_path4, normal_model))

Computing metrics for {} for benign
Computing metrics for {} for adv
(0.9890000000000001, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.005, 0.03, 0.11899999999999998, 0.226, 0.335, 0.47, 0.664, 0.938, 0.9940000000000001])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.994, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.003, 0.036, 0.131, 0.21899999999999997, 0.32, 0.465, 0.672, 0.9280000000000002, 0.997])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.9849999999999999, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.01, 0.039, 0.127, 0.207, 0.331, 0.488, 0.675, 0.91, 0.995])


# BIM

In [37]:
adv_path1 = 'adv samples/IMAGENET/ResNet50/BIM/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/IMAGENET/ResNet50/BIM/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/IMAGENET/ResNet50/BIM/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/IMAGENET/ResNet50/BIM/0.25098039215686274eps.npz'

In [38]:
auc, tpr, fpr = return_auc(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.3494965,
 [0.126, 0.154, 0.192, 0.22, 0.23799999999999996, 0.277, 0.345, 0.6, 0.873],
 [0.002, 0.036, 0.127, 0.205, 0.3, 0.42699999999999994, 0.625, 0.917, 0.997])

In [39]:
print(return_auc(adv_path2, normal_model))
print('----')
print(return_auc(adv_path3, normal_model))
print('----')
print(return_auc(adv_path4, normal_model))

Computing metrics for {} for benign
Computing metrics for {} for adv
(0.4697404999999999, [0.374, 0.379, 0.391, 0.4040000000000001, 0.41600000000000004, 0.442, 0.474, 0.613, 0.863], [0.004, 0.04100000000000001, 0.139, 0.24600000000000002, 0.32899999999999996, 0.482, 0.666, 0.9330000000000002, 0.996])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.5870895, [0.537, 0.539, 0.545, 0.551, 0.554, 0.563, 0.572, 0.682, 0.8539999999999999], [0.003, 0.037, 0.121, 0.22, 0.315, 0.447, 0.653, 0.93, 0.998])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.6162814999999999, [0.585, 0.586, 0.586, 0.588, 0.588, 0.588, 0.602, 0.715, 0.88], [0.002, 0.04, 0.135, 0.221, 0.299, 0.44299999999999995, 0.639, 0.921, 0.99])


# CW

In [40]:
adv_path1 = 'adv samples/IMAGENET/ResNet50/CW/0.15eps.npz'

In [41]:
auc, tpr, fpr = return_auc(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.9513506559031282,
 [0.7214934409687185,
  0.8718466195761857,
  0.9445005045408678,
  0.9616548940464178,
  0.9656912209889001,
  0.9778002018163471,
  0.987891019172553,
  0.9969727547931383,
  0.9989909182643796],
 [0.006,
  0.047,
  0.138,
  0.22699999999999998,
  0.332,
  0.45899999999999996,
  0.644,
  0.915,
  0.991])

# MobileNet Model

In [42]:
# load mobilet model first

def load_imagenet_model():
    model=torchvision.models.mobilenet_v3_small(weights=True).to(device)
    model.to('cuda')
    model.train(False)
    return model

In [43]:
normal_model = load_imagenet_model()
normal_model.to(device)
normal_model.eval()

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [44]:
def return_auc(adv_path, model):
    #logitgaussian3
    k=[1710, 1810,2000, 2510,2810,3000, 3200,3500, 6000]
    l=[6700, 6700,6700, 6700,6700,6700,6700,6700, 6700]
    
    #attrgaussian3
    m=[1200, 1400, 1600, 1750,1900, 2300, 3200, 3500, 6000]
    n=[6600, 6600,6600, 6600,6600,6600,6600,6600,6600]

    
    compute_metrics_benign(adv_path, model)
    compute_metrics_adv(adv_path, model)
    df_cifar = pd.read_csv("adaptive_Benign.csv")
    attr_gaussian3 = df_cifar.iloc[2].values.flatten().tolist()[1:]
    logit_gaussian3 = df_cifar.iloc[6].values.flatten().tolist()[1:]
        
    fpr_results =[]
    for t1,t2,t3,t4 in zip(k,l,m,n):
        FPR = compute_FPR(logit_gaussian3, t1,t2, attr_gaussian3,t3,t4)
        fpr_results.append(FPR/100)
        
    df_pgd_eps1 = pd.read_csv("adaptive_Adv.csv")
    attr_gaussian3_eps1 = df_pgd_eps1.iloc[2].values.flatten().tolist()[1:]
    logit_gaussian3_eps1 = df_pgd_eps1.iloc[6].values.flatten().tolist()[1:]
    
    tpr_results =[]
    for t1,t2,t3,t4 in zip(k,l,m,n):
        TPR = compute_TPR(logit_gaussian3_eps1, t1,t2, attr_gaussian3_eps1,t3,t4)
        tpr_results.append(TPR/100)
    return sklearn.metrics.auc(fpr_results, tpr_results), fpr_results, tpr_results

# FGSM

In [45]:

adv_path1 = 'adv samples/IMAGENET/MobileNet/FGSM/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/IMAGENET/MobileNet/FGSM/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/IMAGENET/MobileNet/FGSM/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/IMAGENET/MobileNet/FGSM/0.25098039215686274eps.npz'

In [46]:
auc, tpr, fpr = return_auc(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.8148525,
 [0.03, 0.06, 0.121, 0.207, 0.308, 0.542, 0.8809999999999999, 0.931, 1.0],
 [0.105, 0.262, 0.512, 0.704, 0.825, 0.948, 0.998, 1.0, 1.0])

In [47]:
print(return_auc(adv_path2, normal_model))
print('----')
print(return_auc(adv_path3, normal_model))
print('----')
print(return_auc(adv_path4, normal_model))

Computing metrics for {} for benign
Computing metrics for {} for adv
(0.9126704999999999, [0.027000000000000003, 0.059, 0.124, 0.20800000000000002, 0.317, 0.559, 0.894, 0.949, 1.0], [0.276, 0.567, 0.815, 0.9230000000000002, 0.971, 0.995, 1.0, 1.0, 1.0])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.960005, [0.031, 0.06, 0.11600000000000002, 0.21600000000000003, 0.313, 0.565, 0.885, 0.9330000000000002, 1.0], [0.762, 0.915, 0.98, 0.998, 0.9990000000000001, 1.0, 1.0, 1.0, 1.0])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.979014, [0.02, 0.057, 0.12, 0.18600000000000003, 0.287, 0.522, 0.8640000000000001, 0.916, 1.0], [0.961, 0.996, 0.9990000000000001, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])


# PGD

In [48]:
adv_path1 = 'adv samples/IMAGENET/MobileNet/PGD/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/IMAGENET/MobileNet/PGD/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/IMAGENET/MobileNet/PGD/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/IMAGENET/MobileNet/PGD/0.25098039215686274eps.npz'

In [49]:
auc, tpr, fpr = return_auc(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.9549650000000001,
 [0.039,
  0.077,
  0.148,
  0.235,
  0.32800000000000007,
  0.568,
  0.8740000000000001,
  0.926,
  1.0],
 [0.985, 0.987, 0.988, 0.99, 0.993, 0.995, 0.997, 0.998, 1.0])

In [50]:
print(return_auc(adv_path2, normal_model))
print('----')
print(return_auc(adv_path3, normal_model))
print('----')
print(return_auc(adv_path4, normal_model))

Computing metrics for {} for benign
Computing metrics for {} for adv
(0.959, [0.04100000000000001, 0.074, 0.14, 0.239, 0.32899999999999996, 0.538, 0.877, 0.929, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.969, [0.031, 0.063, 0.12, 0.221, 0.333, 0.561, 0.892, 0.938, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.965, [0.035, 0.069, 0.13, 0.228, 0.332, 0.579, 0.8759999999999999, 0.937, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])


# BIM

In [51]:
adv_path1 = 'adv samples/IMAGENET/MobileNet/BIM/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/IMAGENET/MobileNet/BIM/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/IMAGENET/MobileNet/BIM/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/IMAGENET/MobileNet/BIM/0.25098039215686274eps.npz'

In [52]:
auc, tpr, fpr = return_auc(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.4813319999999999,
 [0.029000000000000005,
  0.049,
  0.11600000000000002,
  0.212,
  0.31,
  0.53,
  0.883,
  0.94,
  1.0],
 [0.272, 0.277, 0.288, 0.332, 0.363, 0.442, 0.728, 0.809, 0.9990000000000001])

In [53]:
print(return_auc(adv_path2, normal_model))
print('----')
print(return_auc(adv_path3, normal_model))
print('----')
print(return_auc(adv_path4, normal_model))

Computing metrics for {} for benign
Computing metrics for {} for adv
(0.6262220000000001, [0.031, 0.069, 0.131, 0.222, 0.331, 0.55, 0.882, 0.938, 1.0], [0.52, 0.522, 0.525, 0.542, 0.56, 0.621, 0.786, 0.839, 0.9990000000000001])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.763795, [0.034, 0.074, 0.139, 0.21899999999999997, 0.317, 0.531, 0.877, 0.9330000000000002, 1.0], [0.731, 0.731, 0.732, 0.737, 0.742, 0.762, 0.865, 0.905, 1.0])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.8369044999999999, [0.024, 0.05800000000000001, 0.118, 0.217, 0.324, 0.55, 0.867, 0.932, 1.0], [0.802, 0.8030000000000002, 0.8030000000000002, 0.8080000000000002, 0.818, 0.845, 0.921, 0.9469999999999998, 0.9990000000000001])


# CW

In [54]:
adv_path1 = 'adv samples/IMAGENET/MobileNet/CW/0.15eps.npz'

auc, tpr, fpr = return_auc(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.8679844999999999,
 [0.032, 0.072, 0.129, 0.23, 0.33, 0.548, 0.882, 0.935, 1.0],
 [0.533,
  0.572,
  0.657,
  0.861,
  0.926,
  0.9519999999999998,
  0.982,
  0.9940000000000001,
  1.0])